In [14]:
%load_ext autoreload
%autoreload 2

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals
import data_strings
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
import trajectory_analysis.trajectory_headangle as trajectory_headangle
import data_extraction.get_indices as get_indices
import plotting.plot_probability_chose_wall as plot_probability_chose_wall
import analysis.conditioned_player_choice as conditioned_player_choice
import parse_data.identify_filepaths as identify_filepaths

plt.rcParams['figure.dpi'] = 100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Calculation of P(choice | visibility) analysis using the logic from opponent-visibility 

In [15]:
# data_folder = data_strings.DATA_FOLDER
# json_filenames_all_social = data_strings.JSON_FILENAMES_SOCIAL
# json_filenames_all_solo = data_strings.JSON_FILENAMES_SOLO

In [27]:
data_folder = r'D:\Tom\Downloads\pseudonymised_json_files\pseudonymised_json_files'
json_filenames_social, json_filenames_first_solo, json_filenames_second_solo = identify_filepaths.get_all_relative_paths(data_folder=data_folder)
json_filenames_both_solos = [session for pair in zip(json_filenames_first_solo, json_filenames_second_solo) for session in pair]


In [28]:
json_filenames_social

['240913_1\\2024-09-13_11-31-00_YW13_JL13_Social.json',
 '240927_1\\2024-09-27_14-25-20_SH27_EN27_Social.json',
 '241017_1\\2024-10-17_14-28-40_SP17_AW17_Social.json',
 '241017_2\\2024-10-17_16-41-38_ZH17_EM17_Social.json',
 '241112_1\\2024-11-12_13-31-14_KA12_WM12_Social.json',
 '241112_2\\2024-11-12_15-23-24_FA12_SL12_Social.json',
 '241113_1\\2024-11-13_14-18-54_NK13_RD13_Social.json',
 '241113_2\\2024-11-13_15-28-07_YL13_HC13_Social.json',
 '241119_1\\2024-11-19_14-24-49_AV19_XG19_Social.json',
 '241119_2\\2024-11-19_15-22-56_SB19_HH19_Social.json',
 '241120_1\\2024-11-20_14-17-44_JS20_RR20_Social.json',
 '241120_2\\2024-11-20_15-16-21_ZS20_VC20_Social.json',
 '241203_1\\2024-12-03_14-31-51_PO03_NN03_Social.json',
 '241203_2\\2024-12-03_15-27-28_EX03_BC03_Social.json',
 '241210_1\\2024-12-10_14-21-17_TE10_TK10_Social.json',
 '241210_2\\2024-12-10_15-20-11_RK10_RU10_Social.json',
 '241219_1\\2024-12-19_15-28-24_JU19_SY19_Social.json',
 '241220_1\\2024-12-20_17-44-50_KS20_CS20_Social

In [3]:
json_filenames = json_filenames_all_social

In [5]:
df, trial_lists = prepare_data.prepare_data(data_folder, json_filenames, combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\240913_1\2024-09-13_11-31-00_YW13_JL13_Social.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\240927_1\2024-09-27_14-25-20_SH27_EN27_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\241017_1\2024-10-17_14-28-40_SP17_AW17_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\241017_2\2024-10-17_16-41-38_ZH17_EM17_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\241112_1\2024-11-12_13-31-14_KA12_WM12_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\241112_2\2024-11-12_15-23-24_FA12_SS12_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\241113_1\2024-11

In [6]:
df_solo, trial_lists_solo = prepare_data.prepare_data(data_folder, json_filenames_all_solo, combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\240913_1\2024-09-13_11-23-37_YW13_FirstSolo.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\240913_1\2024-09-13_11-53-34_YW13_SecondSolo.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\240913_1\2024-09-13_11-23-50_JL13_FirstSolo.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\240913_1\2024-09-13_11-53-56_JL13_SecondSolo.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: 

In [ ]:
analysis_results = {
    session_id: {
        player_id: {

            # filtered, then flipped-and-rotated trial dataframes
            'filtered_trials': {
                'numerator': filtered_trials_numerator[session_id][player_id],
                'denominator': filtered_trials_denominator[session_id][player_id],
                'numerator_flipped_rotated': None,
                'denominator_flipped_rotated': None
            },

            # trial start positions
            'positions': {
                'positions_numerator': None,
                'positions_denominator': None,
            },

            # grids of count data per trial
            'grids': {
                'grid_numerator': None,
                'grid_denominator': None,
                'grid_ratio': None # ratio of numerator to denominator grids
            }
        }
        for player_id in filtered_trials_numerator[session_id]
    }
    for session_id in filtered_trials_numerator
}

### Choice conditioned on visibility, using conditioned_player_choice.py

#### P(H|H first vis)

In [27]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 0
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = None
solo = False
data_size_cutoff = 6

In [28]:
## All social datasets
(probabilities1,
 numerator1,
  denominator1) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(trial_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 1. 1. 0. 1.]
And the number of valid trials at this step is 33
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0.]
And the number of valid trials at this step is 43


d:\tom\documents\swc\repos\octagon_analysis\trajectory_analysis\trajectory_vectors.py:321: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0.]
And the number of valid trials at this step is 12
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 28
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1.
 0. 0. 1.]
And the number of va

#### P(H|L first vis)

In [29]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = None
solo = False
data_size_cutoff = 6

In [30]:
## All social datasets
(probabilities2,
 numerator2,
  denominator2) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(trial_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=True)

Next session, player_id 0
initial original indices are: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101]
Len 'player choice exists' indices: 82
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 0.]
And the number of valid trials at this step is 33
Len '1 with initial_vis_only False': 33
Probability player chose wall given these conditions: 0.181818